<a href="https://colab.research.google.com/github/Czedros/CSE352-Machine-Learning-Assignments/blob/main/CSE352MidtermMakeupAssignmentPlayable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This is the Playable Version of the MCTS Ultimate Tic-Tac-Toe Game.

Simply Run the code box and the game will start at the bottom.

Moves are made as (big_r big_c small_r small_c) sdparated by spaces.

Ex. Move: 1 1 1 1

Guide (0 0 1 1) would be top left (Main game) center tile (subgame)

Default Iteration Count is 800, can be tweaked if too slow

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math

def draw_ultimate_board(board, small_board_winners=None, last_move=None, highlight=None):
    """
    Visualizes a 9x9 Ultimate Tic-Tac-Toe board with enhancements:
    - Grays out small boards that have been won
    - Draws a large X or O over won small boards
    - Highlights last move in bold
    - Red border for active small board
    """
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 9)

    # Draw grid
    for i in range(10):
        lw = 2 if i % 3 == 0 else 0.5
        ax.axhline(i, color='black', lw=lw, zorder=0)
        ax.axvline(i, color='black', lw=lw, zorder=0)

    # Draw faded backgrounds for won small boards
    if small_board_winners:
        for br in range(3):
            for bc in range(3):
                winner = small_board_winners[br][bc]
                if winner in [1, 2]:
                    rect = plt.Rectangle((bc*3, 6 - br*3), 3, 3, color='gray', alpha=0.6, zorder=1)
                    ax.add_patch(rect)
                    ax.text(bc*3 + 1.5, 7.5 - br*3, 'X' if winner == 1 else 'O',
                            fontsize=80, ha='center', va='center', color='black', alpha=1, zorder=2)

    # Draw pieces
    for row in range(9):
        for col in range(9):
            val = board[row, col]
            if val == 1:
                ax.text(col+0.5, 8.5 - row, 'X', ha='center', va='center', fontsize=16,
                        fontweight='bold' if (last_move == (row, col)) else 'normal', zorder=3)
            elif val == 2:
                ax.text(col+0.5, 8.5 - row, 'O', ha='center', va='center', fontsize=16,
                        fontweight='bold' if (last_move == (row, col)) else 'normal', zorder=3)

    # Highlight active small board
    if highlight:
        hrow, hcol = highlight
        x0 = hcol * 3
        y0 = 6 - hrow * 3
        ax.plot([x0, x0+3], [y0, y0], color='red', lw=4, zorder=4)
        ax.plot([x0, x0+3], [y0+3, y0+3], color='red', lw=4, zorder=4)
        ax.plot([x0, x0], [y0, y0+3], color='red', lw=4, zorder=4)
        ax.plot([x0+3, x0+3], [y0, y0+3], color='red', lw=4, zorder=4)

    plt.show()


class SmallBoard:
    def __init__(self):
        self.grid = np.zeros((3, 3), dtype=int)
        self.winner = 0
        self.active = True

    def play_move(self, row, col, player):
        if self.grid[row, col] != 0 or not self.active:
            return False
        self.grid[row, col] = player
        self.check_winner()
        return True

    def check_winner(self):
        for i in range(3):
            if np.all(self.grid[i, :] == self.grid[i, 0]) and self.grid[i, 0] != 0:
                self._declare_winner(self.grid[i, 0])
                return
            if np.all(self.grid[:, i] == self.grid[0, i]) and self.grid[0, i] != 0:
                self._declare_winner(self.grid[0, i])
                return
        if np.all(np.diag(self.grid) == self.grid[0, 0]) and self.grid[0, 0] != 0:
            self._declare_winner(self.grid[0, 0])
            return
        if np.all(np.diag(np.fliplr(self.grid)) == self.grid[0, 2]) and self.grid[0, 2] != 0:
            self._declare_winner(self.grid[0, 2])
            return
        if np.all(self.grid != 0):
            self._declare_winner(-1)

    def _declare_winner(self, winner):
        self.winner = winner
        self.active = False

class UltimateBoard:
    def __init__(self):
        self.boards = [[SmallBoard() for _ in range(3)] for _ in range(3)]
        self.current_player = 1  # 1 = X, 2 = O
        self.next_board = None
        self.global_winner = 0
        self.last_move = None

    def play_move(self, big_r, big_c, small_r, small_c):
        board = self.boards[big_r][big_c]
        if self.next_board and (big_r, big_c) != self.next_board:
            print("You must play in board", self.next_board)
            return False
        if not board.play_move(small_r, small_c, self.current_player):
            print("Invalid move.")
            return False

        next_r, next_c = small_r, small_c
        if self.boards[next_r][next_c].active:
            self.next_board = (next_r, next_c)
        else:
            self.next_board = None

        self.check_global_winner()
        self.current_player = 3 - self.current_player
        self.last_move = (big_r, big_c, small_r, small_c)
        return True

    def check_global_winner(self):
        grid = np.array([[b.winner if b.winner > 0 else 0 for b in row] for row in self.boards])
        for i in range(3):
            if np.all(grid[i, :] == grid[i, 0]) and grid[i, 0] != 0:
                self.global_winner = grid[i, 0]
                return
            if np.all(grid[:, i] == grid[0, i]) and grid[0, i] != 0:
                self.global_winner = grid[0, i]
                return
        if np.all(np.diag(grid) == grid[0, 0]) and grid[0, 0] != 0:
            self.global_winner = grid[0, 0]
            return
        if np.all(np.diag(np.fliplr(grid)) == grid[0, 2]) and grid[0, 2] != 0:
            self.global_winner = grid[0, 2]
            return

    def get_legal_moves(self):
        moves = []
        for big_r in range(3):
            for big_c in range(3):
                board = self.boards[big_r][big_c]
                if self.next_board and (big_r, big_c) != self.next_board:
                    continue
                if board.active:
                    for i in range(3):
                        for j in range(3):
                            if board.grid[i, j] == 0:
                                moves.append((big_r, big_c, i, j))
        return moves

    def display(self):
        def cell_symbol(val):
            return '.' if val == 0 else ('X' if val == 1 else 'O')
        for big_r in range(3):
            for r in range(3):
                line = ''
                for big_c in range(3):
                    board = self.boards[big_r][big_c]
                    line += ' '.join(cell_symbol(board.grid[r, c]) for c in range(3)) + ' | '
                print(line)
            print('-' * 20)

def ultimate_board_to_board(game, highlight=None):
    board_array = np.zeros((9, 9), dtype=int)
    small_board_winners = [[game.boards[r][c].winner for c in range(3)] for r in range(3)]
    last_move = None
    if hasattr(game, 'last_move') and game.last_move:
        br, bc, sr, sc = game.last_move
        last_move = (br*3 + sr, bc*3 + sc)

    for br in range(3):
        for bc in range(3):
            sb = game.boards[br][bc].grid
            board_array[br*3:(br+1)*3, bc*3:(bc+1)*3] = sb

    if highlight is None and hasattr(game, 'next_board') and not game.global_winner:
        highlight = game.next_board

    draw_ultimate_board(board_array, small_board_winners=small_board_winners,
                        last_move=last_move, highlight=highlight)


def play_game(agent_X=None, agent_O=None):
    game = UltimateBoard()

    while not game.global_winner and game.get_legal_moves():
        print("\nCurrent Turn: Player {}".format("X" if game.current_player == 1 else "O"))
        ultimate_board_to_board(game)

        if game.next_board:
            print("You must play in small board:", game.next_board)
        else:
            print("You may play in any active board.")

        # Determine agent for this turn
        agent = agent_X if game.current_player == 1 else agent_O

        if agent is None:
            # Human move
            move = input("Enter your move (big_r big_c small_r small_c): ").split()
            try:
                move = tuple(map(int, move))
                if len(move) != 4 or not game.play_move(*move):
                    print("Try again.")
            except:
                print("Invalid input.")
        else:
            # Bot move
            move = agent(game)
            print(f"Bot plays: {move}")
            game.play_move(*move)

    print("\nFinal Result:")
    ultimate_board_to_board(game)
    if game.global_winner:
        print("Winner: Player", "X" if game.global_winner == 1 else "O")
    else:
        print("Game ended in a draw.")

class EnhancedMCTSNode:
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = game_state
        self.parent = parent
        self.move = move
        self.children = []
        self.visits = 0
        self.total_reward = 0.0
        self.prior = 1.0
        self.untried_moves = self._prioritized_moves(game_state.get_legal_moves())

    def _prioritized_moves(self, moves):
        prioritized = []
        for move in moves:
            small_r, small_c = move[2], move[3]
            priority = 1.0
            if (small_r, small_c) == (1, 1):  # Center
                priority = 1.5
            elif (small_r % 2 == 0) and (small_c % 2 == 0):  # Corners
                priority = 1.2
            prioritized.append((move, priority))
        # Sort moves to prioritize stronger ones first
        prioritized.sort(key=lambda x: x[1], reverse=True)
        return prioritized

    def is_fully_expanded(self):
        return len(self.untried_moves) == 0

    def select_child(self, c_puct=1.5):
        total_n = math.sqrt(self.visits + 1)
        best_score = -float('inf')
        best_child = None

        for child in self.children:
            if child.visits == 0:
                q_value = 0
            else:
                q_value = child.total_reward / child.visits

            puct_value = c_puct * child.prior * total_n / (1 + child.visits)
            score = q_value + puct_value

            if score > best_score:
                best_score = score
                best_child = child
        return best_child

    def __repr__(self):
        return f"Node(move={self.move}, visits={self.visits}, reward={self.total_reward})"

def copy_ultimate_board(original):
    new_board = UltimateBoard()
    new_board.current_player = original.current_player
    new_board.next_board = original.next_board
    new_board.global_winner = original.global_winner

    for i in range(3):
        for j in range(3):
            orig_sb = original.boards[i][j]
            new_sb = SmallBoard()
            new_sb.grid = orig_sb.grid.copy()
            new_sb.winner = orig_sb.winner
            new_sb.active = orig_sb.active
            new_board.boards[i][j] = new_sb
    return new_board

def calculate_reward(final_state, root_player):
    if final_state.global_winner == root_player:
        return 1.0
    if final_state.global_winner == -1:
        return 0.4
    if final_state.global_winner == 3 - root_player:
        return -1.0

    score = 0
    small_board_counts = np.zeros((3,3))
    for i in range(3):
        for j in range(3):
            if final_state.boards[i][j].winner == root_player:
                score += 0.1
                small_board_counts[i,j] = 1
            elif final_state.boards[i][j].winner == 3 - root_player:
                score -= 0.1
                small_board_counts[i,j] = -1

    for i in range(3):
        row_sum = np.sum(small_board_counts[i,:])
        col_sum = np.sum(small_board_counts[:,i])
        if abs(row_sum) == 3 or abs(col_sum) == 3:
            score += 0.3 * np.sign(row_sum if abs(row_sum)==3 else col_sum)

    return score / (1 + abs(score))

def enhanced_simulation(game_state, max_depth=30):
    sim_game = copy_ultimate_board(game_state)
    depth = 0

    while sim_game.global_winner == 0 and sim_game.get_legal_moves() and depth < max_depth:
        moves = sim_game.get_legal_moves()
        best_moves = []

        for move in moves:
            temp_game = copy_ultimate_board(sim_game)
            temp_game.play_move(*move)

            # Check for immediate global win
            if temp_game.global_winner == sim_game.current_player:
                return temp_game

            # Small board win heuristic
            big_r, big_c = move[0], move[1]
            if temp_game.boards[big_r][big_c].winner == sim_game.current_player:
                best_moves.append(move)

        if best_moves:
            move = random.choice(best_moves)
        else:
            scored = []
            for move in moves:
                sr, sc = move[2], move[3]
                if (sr, sc) == (1, 1):
                    score = 3
                elif sr % 2 == 0 and sc % 2 == 0:
                    score = 2
                else:
                    score = 1
                scored.append((move, score))

            max_score = max(score for _, score in scored)
            fallback = [m for m, s in scored if s == max_score]
            move = random.choice(fallback)

        sim_game.play_move(*move)
        depth += 1

    return sim_game

def enhanced_mcts(ultimate_board, iterations=800):
    root = EnhancedMCTSNode(copy_ultimate_board(ultimate_board))

    for _ in range(iterations):
        node = root
        path = [node]

        while not node.game_state.global_winner and node.is_fully_expanded() and node.children:
            node = node.select_child()
            path.append(node)

        if node.untried_moves and not node.game_state.global_winner:
            move, prior = node.untried_moves.pop()
            new_game = copy_ultimate_board(node.game_state)
            new_game.play_move(*move)

            if new_game.global_winner == node.game_state.current_player:
                reward = 1.0
                child = EnhancedMCTSNode(new_game, parent=node, move=move)
                child.prior = prior
                node.children.append(child)
                path.append(child)
                for n in path:
                    n.visits += 1
                    n.total_reward += reward
                continue

            child = EnhancedMCTSNode(new_game, parent=node, move=move)
            child.prior = prior
            node.children.append(child)
            path.append(child)
            node = child

        final_state = enhanced_simulation(node.game_state)
        reward = calculate_reward(final_state, root.game_state.current_player)

        for n in path:
            n.visits += 1
            n.total_reward += reward

    if not root.children:
        return random.choice(ultimate_board.get_legal_moves())

    best_child = max(root.children, key=lambda c: c.visits)
    sure_win = [c for c in root.children if c.visits >= 10 and (c.total_reward / c.visits) == 1.0]
    if sure_win:
        return random.choice(sure_win).move
    return best_child.move
def enhanced_wrapper(game):
  return enhanced_mcts(game, 800)

play_game(None, enhanced_wrapper)